### **Домашнее задание: Выбор модели для бинарной классификации**

**Цель**: Провести отбор признаков и настройку гиперпараметров нескольких моделей для бинарной классификации разными методами. Сравнить результаты и выбрать лучшую модель.

Задание считается выполненным успешно, если будет обучено по крайней мере три модели, среди которых выбрана лучшая по тестовым метрикам.

Ноутбуки направить на почту simon.ilishaev@gmail.com. В теме письма - [ML в Рисках]


#### **Данные и начальная настройка**  
1. [Загрузите датасет](https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset) (числовые и категориальные признаки, бинарная целевая переменная).  
2. Сделайте **стратифицированное разделение на train-test** (например, 70-30). **Тестовый набор** будет использоваться **только для финальной оценки модели**.

#### Подход с разделением на train-validation  
1. Разделите ещё раз **обучающую выборку (train)** на train-validation** (например, 80-20).  
2. Проведите **отбор признаков с помощью фильтрационных методов** на **train-подвыборке**.  
3. Настройте гиперпараметры (например, `C` для логистической регрессии, `max_depth` для дерева решений и т. д.) на **валидационной выборке**.  
4. **Опционально**: Используйте **Differential Evolution из Scipy** для оптимизации гиперпараметров логистической регрессии.  

#### Подход с кросс-валидацией  
1. Используйте **кросс-валидацию (CV)** для **отбора признаков и настройки гиперпараметров**.  
2. Реализуйте **GridSearchCV** для перебора гиперпараметров.  
3. **Опционально**: Используйте **Optuna** с **многокритериальной оптимизацией** (максимизация ROC-AUC и Precision-Recall AUC).  
4. **Опционально**: Визуализируйте **Парето-фронт** для испытаний Optuna.  

#### **Финальная оценка моделей**  
1. Оцените все настроенные модели на **тестовом наборе** (ROC-AUC, Precision-Recall AUC, F1-score).  
2. **Выберите лучшую модель** на основе тестовых метрик.  

### **Модели для использования**  
- Логистическая регрессия (`LogisticRegression`)  
- Дерево решений (`DecisionTreeClassifier`)  
- Случайный лес (`RandomForestClassifier`)
- ...

### Документация

[Scikit-Learn Cross-Validation](https://scikit-learn.org/stable/modules/cross_validation.html)

[Category Encoders](https://contrib.scikit-learn.org/category_encoders/)

[Grid Search](https://scikit-learn.org/stable/modules/grid_search.html)

[Optuna example](https://github.com/optuna/optuna-examples/blob/main/sklearn/sklearn_simple.py)

[Pareto front](https://optuna.readthedocs.io/en/stable/reference/visualization/generated/optuna.visualization.plot_pareto_front.html#sphx-glr-reference-visualization-generated-optuna-visualization-plot-pareto-front-py)

[Scikit-Leaern Pipeline](https://scikit-learn.org/stable/modules/compose.html)

[Differential Evolution](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html)
---

In [ ]:
%%bash
pip install ucimlrepo
pip install category_encoders
pip install optuna

In [29]:
# библиотеки, которые могут понадобиться для выполнения задания
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score
from category_encoders import TargetEncoder
from scipy.optimize import differential_evolution
import optuna
from optuna.visualization import plot_pareto_front
import matplotlib.pyplot as plt

In [31]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
secondary_mushroom = fetch_ucirepo(id=848)

# data (as pandas dataframes)
X = secondary_mushroom.data.features
y = secondary_mushroom.data.targets

# раскомментируйте, чтобы посмотреть метаданные набора данных
# # metadata
# print(secondary_mushroom.metadata)

# # variable information
# print(secondary_mushroom.variables)

In [32]:
# target: p - poisonous (ядовитые), e - edible(съедобные)
y = y['class'].map({'p': 1, 'e': 0})

In [ ]:
# Разделение на train-test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Категориальные признаки
cat_cols = list(X.select_dtypes('object').columns)
print(cat_cols)
# Численные признаки
num_cols = [col for col in X.columns if col not in cat_cols + ["target"]]
print(num_cols)

In [ ]:
# Подсказка, соберите конвейер из нескольких компонент
pipeline = Pipeline([
    ("encoder", TargetEncoder(cols=cat_cols)),
    ("selector", SelectKBest(score_func=f_classif)),
    ("model", LogisticRegression(max_iter=1000))
])

# Пример с логистической регрессией
# Настройка через GridSearchCV с RepeatedKFold
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
param_grid = {
    "selector__k": [5, 10, 15],
    "model__C": [0.01, 0.1, 1, 10]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring="roc_auc", n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Оценка на тесте
test_roc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
print(f"Test ROC-AUC (GridSearch): {test_roc:.3f}")